In [ ]:
import pandas as pd 
import dataset
import network
from transformers import AutoTokenizer
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
 
dataset = pd.read_csv('/kaggle/input/sentiment/Combined Data.csv')
dataset = dataset.sample(n=5000).reset_index(drop=True)
train, val, sentiment = dataset.get_dataloader(dataset, AutoTokenizer.from_pretrained('distilroberta-base'), 32)

model = network.create_model(sentiment).to(device)

early_stop = network.EarlyStopping(5, 0.01)
optimizer = AdamW(model.parameters(), lr=1e-3)
loss = CrossEntropyLoss()
epochs = 10

min_cross, max_acc = None, 0
for epoch in range(epochs):
    network.train_loop(train, model, loss, optimizer, 32, device)
    cross, acc = network.validation_loop(val, model, loss, device)
    print(f'Sul validation set:\nCross entropy: {cross}, Accuracy: {acc}')
    if min_cross == None or min_cross > cross:
        min_cross = cross
        max_acc = acc
        model.roberta.save_pretrained("./my_model")
        
    stop = early_stop(cross)
    if stop:
        break
    
print(f'La cross validation finale è {min_cross}, l\'accuratezza è {max_acc}')




c:\Users\rosil\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rosil\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 